In [6]:
import numpy as np
import torch
from DVR.core import *
import matplotlib.pyplot as plt

N = 5
k = 3
n = np.array([N, 0, 0])
R0 = np.array([3, 3, 7.2])
dvr = DVR(n,
          R0=R0,
          avg=1,
          sparse=True,
          symmetry=True)
dvr.p *= dvr.n != 0
dvr.dx *= dvr.n != 0

T = dvr.Tmat()
V, no = dvr.Vmat()
# for i in range(3):
#     print(T[i])
# print(V)
if dvr.verbosity > 2:
    print("H_op: n={} dx={}w p={} {} operator constructed.".format(
        dvr.n[dvr.nd], dvr.dx[dvr.nd], dvr.p[dvr.nd], dvr.model))

t0 = time()
N = np.prod(no)

if dvr.verbosity > 2:
    print('H_solver: diagonalize sparse hermitian matrix.')
H = LinearOperator([torch.from_numpy(T[i]).requires_grad_()
                   for i in range(len(T))], torch.Tensor(V).requires_grad_(), no)
e1, w1 = symeig(H, k)
def applyH(psi) -> np.ndarray: return dvr.H_op(T, V, no, psi)


H = ssla.LinearOperator((N, N), matvec=applyH)
e2, w2 = ssla.eigsh(H, k, which='SA')
print(e1.detach().numpy() - e2)
print(abs(w1.detach().numpy()) - abs(w2))


DVR: dx=[0.6]w is set.
DVR: n=[5] is set.
DVR: R0=[3.]w is set.
['x']-reflection symmetry is used.
param_set: trap parameter V0=104.52kHz w=1000nm
[ 2.22044605e-16 -5.55111512e-17  2.60208521e-18]
[[-3.33066907e-16  2.22044605e-16  1.09287579e-16]
 [ 1.80411242e-16  4.44089210e-16  1.24900090e-16]
 [ 9.71445147e-17  1.38777878e-17 -3.33066907e-16]
 [-1.63064007e-16  4.85722573e-17  4.44089210e-16]
 [ 1.24466409e-16  1.73472348e-16 -3.33066907e-16]
 [ 8.76035355e-17  8.67361738e-17 -2.77555756e-16]]
